In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer


from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from collections import Counter
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report,roc_curve, roc_auc_score
import warnings

In [2]:
df = pd.read_csv('../65 Years of Weather Data Bangladesh (1948 - 2013).csv')
df.head()

,No,Station_Names,YEAR,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period
0,0,Barisal,1949,1,29.4,12.3,0.0,68.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1949.01
1,1,Barisal,1950,1,30.0,14.1,0.0,77.0,0.453704,0.8,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1950.01
2,2,Barisal,1951,1,28.2,12.3,0.0,77.0,0.453704,0.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1951.01
3,3,Barisal,1952,1,26.6,12.3,2.0,77.0,0.453704,1.0,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1952.01
4,4,Barisal,1953,1,30.0,13.3,10.0,75.0,0.453704,1.6,7.831915,41950,536809.8,510151.9,22.7,90.36,4,1953.01


In [3]:
# Players to be selected
li = ['Dinajpur','Syedpur','Rangpur','Bogra','Mymensingh']

df_new=df[df.Station_Names.isin(li)]
df_new

,No,Station_Names,YEAR,Month,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,Station_Number,X_COR,Y_COR,LATITUDE,LONGITUDE,ALT,Period
1356,1356,Bogra,1948,1,25.800000,11.700000,8.140625,74.218750,0.875926,1.1,7.296226,41883,435303.7,751187.5,24.88,89.360000,20,1948.01
1357,1357,Bogra,1949,1,27.500000,11.900000,8.140625,74.218750,0.875926,0.7,7.296226,41883,435303.7,751187.5,24.88,89.360000,20,1949.01
1358,1358,Bogra,1950,1,25.800000,11.700000,0.000000,68.000000,0.875926,0.8,7.296226,41883,435303.7,751187.5,24.88,89.360000,20,1950.01
1359,1359,Bogra,1951,1,25.800000,11.700000,5.000000,71.000000,0.875926,0.7,7.296226,41883,435303.7,751187.5,24.88,89.360000,20,1951.01
1360,1360,Bogra,1952,1,33.200000,12.300000,0.000000,66.000000,0.875926,1.0,7.296226,41883,435303.7,751187.5,24.88,89.360000,20,1952.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19651,19654,Syedpur,2009,12,28.342857,12.852381,5.333333,76.428571,1.476190,0.6,5.066667,41858,0.0,0.0,25.75,88.916667,0,2009.12
19652,19655,Syedpur,2010,12,28.342857,12.852381,5.333333,76.428571,1.476190,1.2,5.066667,41858,0.0,0.0,25.75,88.916667,0,2010.12
19653,19629,Syedpur,2011,12,30.400000,12.300000,0.000000,82.000000,4.600000,2.0,4.600000,41858,0.0,0.0,25.75,88.916667,0,2011.12
19654,19630,Syedpur,2012,12,28.400000,12.300000,0.000000,86.000000,2.400000,2.7,2.200000,41858,0.0,0.0,25.75,88.916667,0,2012.12


In [4]:
df_new=df_new.drop(labels=["No","Station_Names","YEAR","Month","ALT","Station_Number"], axis=1)
df_new.head()

,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Wind_Speed,Cloud_Coverage,Bright_Sunshine,X_COR,Y_COR,LATITUDE,LONGITUDE,Period
1356,25.8,11.7,8.140625,74.21875,0.875926,1.1,7.296226,435303.7,751187.5,24.88,89.36,1948.01
1357,27.5,11.9,8.140625,74.21875,0.875926,0.7,7.296226,435303.7,751187.5,24.88,89.36,1949.01
1358,25.8,11.7,0.000000,68.00000,0.875926,0.8,7.296226,435303.7,751187.5,24.88,89.36,1950.01
1359,25.8,11.7,5.000000,71.00000,0.875926,0.7,7.296226,435303.7,751187.5,24.88,89.36,1951.01
1360,33.2,12.3,0.000000,66.00000,0.875926,1.0,7.296226,435303.7,751187.5,24.88,89.36,1952.01


In [5]:
z = df_new["Wind_Speed"]

df_new = df_new.drop(labels=["Wind_Speed","X_COR","Y_COR","Bright_Sunshine"], axis=1)
df_new['Wind_Speed'] = z
df_new.head()

,Max_Temp,Min_Temp,Rainfall,Relative_Humidity,Cloud_Coverage,LATITUDE,LONGITUDE,Period,Wind_Speed
1356,25.8,11.7,8.140625,74.21875,1.1,24.88,89.36,1948.01,0.875926
1357,27.5,11.9,8.140625,74.21875,0.7,24.88,89.36,1949.01,0.875926
1358,25.8,11.7,0.000000,68.00000,0.8,24.88,89.36,1950.01,0.875926
1359,25.8,11.7,5.000000,71.00000,0.7,24.88,89.36,1951.01,0.875926
1360,33.2,12.3,0.000000,66.00000,1.0,24.88,89.36,1952.01,0.875926


In [6]:
X = df_new.iloc[:,:-1]
Y = df_new.iloc[:,-1]

In [7]:
from sklearn.model_selection import train_test_split
trainX, testX, trainY, testY = train_test_split(X, Y, test_size = 0.2)

In [8]:
# Standard Scaler

from sklearn.preprocessing import StandardScaler

sc=StandardScaler()
scaler = sc.fit(trainX)
trainX_scaled = scaler.transform(trainX)
testX_scaled = scaler.transform(testX)

In [11]:
from xgboost import XGBRegressor

RegModel=XGBRegressor(max_depth=3, learning_rate=0.1, n_estimators=500, objective='reg:linear', booster='gbtree')
XGB=RegModel.fit(trainX_scaled,trainY)
prediction=XGB.predict(testX_scaled)

[19:58:48] WARNING: C:/Users/administrator/workspace/xgboost-win64_release_1.6.0/src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.


In [12]:
df_temp = pd.DataFrame({'Actual': testY, 'Predicted': prediction})
df_temp.head()

,Actual,Predicted
14273,0.300000,0.283925
1951,0.809259,0.952703
13950,1.411111,1.505389
13746,1.200000,1.035798
1898,0.800000,1.080711


In [34]:
#Measuring Goodness of fit in Training data
from sklearn import metrics
print('R2 Value:',metrics.r2_score(testY, prediction))

print('Mean Absolute Error:', metrics.mean_absolute_error(testY, prediction))  
print('Mean Squared Error:', metrics.mean_squared_error(testY, prediction))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(testY, prediction)))

R2 Value: 0.8037533144137881
Mean Absolute Error: 0.23068230017265462
Mean Squared Error: 0.11407345551042533
Root Mean Squared Error: 0.33774762102852085
